### settings.py

In [1]:
from sqlalchemy import types


arrivals_tomorrow_dtypes = {
    'dep_airport': types.NVARCHAR(length=255),
    'sched_arr_loc_time': types.DateTime(),
    'arr_terminal': types.INTEGER(),
    'status': types.NVARCHAR(length=255),
    'airline': types.NVARCHAR(length=255),
    'aircraft': types.NVARCHAR(length=255),
    'arr_airport_icoa': types.NVARCHAR(length=255),
    'city': types.NVARCHAR(length=255),
    'country': types.NVARCHAR(length=255),
}

weather_data_dtypes = {
'clouds': types.INTEGER(),
'visibility': types.INTEGER(),
'city': types.NVARCHAR(length=255),
'country': types.NVARCHAR(length=255),
'latitude': types.Float(precision=4, asdecimal=True),
'longitude': types.Float(precision=4, asdecimal=True),
'wind_speed': types.Float(precision=4, asdecimal=True),
'wind_deg': types.INTEGER(),
'wind_gust': types.Float(precision=4, asdecimal=True),
'temp': types.Float(precision=4, asdecimal=True),
'feels_like': types.Float(precision=4, asdecimal=True),
'temp_min': types.Float(precision=4, asdecimal=True),
'temp_max': types.Float(precision=4, asdecimal=True),
'pressure': types.INTEGER(),
'sea_level': types.INTEGER(),
'grnd_level': types.INTEGER(),
'humidity': types.INTEGER(),
'description': types.NVARCHAR(length=255),
'snow': types.Float(precision=4, asdecimal=True),
'rain': types.Float(precision=4, asdecimal=True),
'datetime': types.DateTime(),
}


### helpers.py

In [2]:
import requests
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, engine


def get_weather_forecast_for_tomorrow(city, country_code, rapid_api_key):
    owm_url = "https://community-open-weather-map.p.rapidapi.com/forecast"
    querystring = {"q": f"{city},{country_code}"}
    headers = {
        'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }

    response = requests.request("GET", owm_url, headers=headers, params=querystring)
    print('owm response: ', response.json()['cod'])
    
    return response

def extract_weather_data(weather_json):
    weather_data = weather_json.json()['list'].copy()
    for item in weather_data:
        
        # Extract city data 
        item['city'] = weather_json.json()['city']['name']
        item['country'] = weather_json.json()['city']['country']
        item['latitude'] = weather_json.json()['city']['coord']['lat']
        item['longitude'] = weather_json.json()['city']['coord']['lon']
        
        # Extract data from nested dictionaries
        for k, v in item['wind'].items():
            item['wind_'+k] = v
        for k, v in item['main'].items():
            item[k] = v
        for k, v in item['weather'][0].items():
            item[k] = v
        item['clouds'] = item['clouds']['all']
        
        # Extract data which may not exist
        try: 
            item['snow'] = item['snow']['3h']
        except: 
            item['snow'] = None
        try: 
            item['rain'] = item['rain']['3h']
        except: 
            item['rain'] = None
        
        # Transform units
        item['datetime'] = datetime.datetime.strptime(item['dt_txt'], "%Y-%m-%d %H:%M:%S")
        item['temp'] = item['temp'] - 273.15
        item['feels_like'] = item['feels_like'] - 273.15
        item['temp_min'] = item['temp_min'] - 273.15
        item['temp_max'] = item['temp_max'] - 273.15
        
        # Remove unnecessary data
        item.pop('wind')
        item.pop('main')
        item.pop('weather')
        item.pop('dt')
        item.pop('dt_txt')
        item.pop('sys')
        item.pop('icon')
        item.pop('id')
        item.pop('temp_kf')
        item.pop('pop')
    
    return pd.DataFrame(weather_data)

def get_airport_arrivals_for_tomorrow(airport_code, rapid_api_key):
    
    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
    year = tomorrow.strftime('%Y')
    month = tomorrow.strftime('%m')
    day = tomorrow.strftime('%d')
    
    aero_data_box_url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_code}/{year}-{month}-{day}T00:00/{year}-{month}-{day}T11:59"
    
    querystring = {"direction":"Arrival", "withCodeshared":"true", "withLocation":"false"}
    
    headers = {
        'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    response = requests.request("GET", aero_data_box_url, headers=headers, params=querystring)
    
    return response

def extract_flight_info(arrivals_response, city, country_code, airport_code):
    
    flight_data = []
    
    for flight in arrivals_response.json()['arrivals']:
        
        try: terminal = int(flight['movement']['terminal'])
        except: terminal = None
        
        flight_data.append({
            'dep_airport': flight['movement']['airport']['name'],
            'sched_arr_loc_time': datetime.datetime.strptime(flight['movement']['scheduledTimeLocal'], "%Y-%m-%d %H:%M+%S:%f"),
            'arr_terminal': terminal,
            'status': flight['status'],
            'airline': flight['airline']['name'],
            'aircraft': flight['aircraft']['model'],
            'arr_airport_icoa': airport_code,
            'city': city,
            'country': country_code,
        })
                      
    return pd.DataFrame(flight_data)

class DatabaseInterface:
    def __init__(self,
                 db_name,
                 user,
                 password,
                 host='localhost',
                 port=3306,
                 driver='mysql+pymysql'):
        
        
        mysql_engine = create_engine(f'{driver}://{user}:{password}@{host}')
        mysql_engine.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        self.db_engine = create_engine(f"{driver}://{user}:{password}@{host}:{port}/{db_name}")

    def insert_data(self, df, table_name, dtype):
        df.to_sql(table_name, self.db_engine, if_exists='append', index=False, dtype=dtype)
        
    def close_connection(self):
        self.db_engine.dispose()


### lambda_function.py

In [3]:
# from helpers import get_weather_forecast_for_tomorrow, extract_weather_data, get_airport_arrivals_for_tomorrow, extract_flight_info, DatabaseInterface
# from settings import rapid_api_key, aws_mysql_host, aws_mysql_user, aws_mysql_password, arrivals_tomorrow_dtypes, weather_data_dtypes

def lambda_handler(event, context):

    airport_code='EDDB'
    city = 'Berlin'
    country_code = 'DE'
    
    print("Getting weather response")
    weather_response = get_weather_forecast_for_tomorrow(city, country_code, rapid_api_key)
    print("Extracting weather data")
    weather_data = extract_weather_data(weather_response)
    
    print("Getting flight response")
    arrivals_response = get_airport_arrivals_for_tomorrow(airport_code, rapid_api_key)
    print("Extracting flight data")
    arrivals_tomorrow = extract_flight_info(arrivals_response, city, country_code, airport_code)
    print("Initializing database interface")
    
    dbi = DatabaseInterface(db_name='wbs_gans_project', 
                            user=aws_mysql_user, 
                            password=aws_mysql_password, 
                            host=aws_mysql_host)
    
    print("Inserting flight data")
    dbi.insert_data(arrivals_tomorrow, 'arrivals_tomorrow', dtype=arrivals_tomorrow_dtypes)
    print("Inserting weather data")
    dbi.insert_data(weather_data, 'weather_forecast', dtype=weather_data_dtypes)
    print("Closing connection")
    dbi.close_connection()
    print("Done")

In [ ]:
event = None
context = None
lambda_function(event, context)